In [1]:
import warnings
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import polars as pl 
from tqdm import tqdm 
import implicit
import joblib
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
%matplotlib inline

In [10]:
# from sklearn.preprocessing import OrdinalEncoder
# import joblib

# oe_u = joblib.load("models\ordinal_enc_users.joblib")

# user_id_enc = oe_u.transform(categories["user_id"].to_numpy().reshape(-1,1))
# categories.with_columns(pl.Series("user_id", user_id_enc).map_elements(lambda x: x[0])).write_parquet('data\\users_relevant_categories_encoded\\1.parquet')


In [2]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from scipy import sparse
import joblib



unknown = joblib.load("unknown_ids.joblib")
encoded = joblib.load("encoded_ids.joblib")
R = sparse.load_npz("data\\full_matrix2.npz")
le_i = joblib.load("models\label_enc_items.joblib")
model = implicit.als.AlternatingLeastSquares()
model = model.load("models\\als_model_50_20_test.npz")
categories_enc = pl.scan_parquet("data\\users_relevant_categories_encoded\*.parquet").collect()
catalog2items = pl.scan_parquet("data\catalog2items.parquet").collect()
catalog2items = (catalog2items.with_columns(pl.Series("item_id", 
                                                      [le_i.transform(catalog2items["item_id"][i]) for i in range(len(catalog2items))]))
                                                                                            .to_pandas().set_index("catalogid"))
# catalog2items = ((catalog2items.with_columns(pl.Series("item_id", 
#                                                       catalog2items["item_id"].to_numpy())))
#                                                                               .to_pandas().set_index("catalogid"))
catalog2items_dict = catalog2items.to_dict()["item_id"]
user2catalogs = (categories_enc.group_by("user_id")
                             .agg(pl.col("relevant_catalogid").unique()))
user2catalogs = user2catalogs.to_pandas().set_index("user_id").map(lambda x: x[0])
user2catalogs_dict = user2catalogs.to_dict()["relevant_catalogid"]

In [ ]:
# encoded = encoded[:1000]

In [3]:
from sklearn.preprocessing import LabelEncoder


def get_items_for_catalog(catalogid):
    # res = catalog2items_dict.get(catalogid)
    # return res[:min(len(res), 10_000)]
    return catalog2items_dict.get(catalogid)


def get_item_candidates_for_user(user_id, R_row):
    catalogs =  user2catalogs_dict[user_id]
    catalogs = catalogs[:min(len(catalogs),30)]
    
    candidates = np.concatenate([catalog2items_dict.get(c, np.empty(0, dtype=np.int32)) for c in catalogs])
    candidates = np.unique(candidates) 


    return np.setdiff1d(candidates, R_row.indices, assume_unique=True)


def make_prediction(user_emb:np.array, item_candidates, item_candidates_emb:np.array):
    scores = item_candidates_emb @ user_emb  
    top_idx = np.argpartition(scores, -100)[-100:]
    top_idx = top_idx[scores[top_idx].argsort()[::-1]]

    return item_candidates[top_idx]


def predict_batch(R_trunc, batch_user_ids):
    results = {}
    item_factors = model.item_factors
    user_factors = model.user_factors
    for i in range(len(batch_user_ids)):
        user = batch_user_ids[i]
        candidates = get_item_candidates_for_user(user, R_trunc[i])

        item_candidates_emb = item_factors[candidates]
        res = make_prediction(user_factors[user], candidates, item_candidates_emb)
        results[user] = res
    return results


In [ ]:
# catalog2items = pl.scan_parquet("data\ml_ozon_recsys_train_final_apparel_items_data\*.parquet").select(["item_id", "catalogid"]).group_by("catalogid").agg(pl.col("item_id").unique()).collect()
# catalog2items.write_parquet("data\catalog2items.parquet")

In [ ]:
# import cProfile
# def chunks(lst, batch_size):
#     for i in range(0, len(lst), batch_size):
#         yield lst[i:i + batch_size]


# batch_size = 300
# all_results = {}

# for batch_users in tqdm(chunks(encoded, batch_size)):
#     # print(cProfile.run("predict_batch(R[batch_users], batch_users)"))
#     predict_batch(R[batch_users], batch_users)
#     break

0it [00:00, ?it/s]

(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)
(100,)


0it [00:02, ?it/s]

(100,)
(100,)
(100,)


KeyboardInterrupt: 

In [4]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
import os 

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"

def chunks(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]


batch_size = 300    
all_results = {}
print("total iters = ", 470000 / batch_size)

# for batch_users in tqdm(chunks(encoded, batch_size)):
    # all_results.update(predict_batch2( R[batch_users], batch_users,  le_i, None))
iterations = 0
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for batch_users in tqdm(chunks(encoded, batch_size)):
        futures.append(
            executor.submit(
                predict_batch, R[batch_users], batch_users
            )
        )

    for future in tqdm(as_completed(futures)):
        iterations += 1
        all_results.update(future.result())
        # if iterations % 100 == 0:
        #     df = pd.DataFrame({
        #         'values': list(all_results.values())
        #     }, index=list(all_results.keys()))
        #     df.to_csv(f'output_2_{iterations}.csv')

# with ProcessPoolExecutor(max_workers=6) as executor:
#     futures = {executor.submit(predict_batch, R[batch], batch, le_i, None): batch
#                for batch in tqdm(chunks(encoded, batch_size))}

#     for i, future in tqdm(enumerate(as_completed(futures), 1)):
#         all_results.update(future.result())
#         if i % 300 == 0:
#             pd.DataFrame.from_dict(all_results, orient="index").to_csv(f"output2_{i}.csv")


df = pd.DataFrame({
    'values': list(all_results.values())
}, index=list(all_results.keys()))
df.to_csv('output3.csv')
all_results

total iters =  1566.6666666666667


1568it [00:29, 52.79it/s] 
1568it [3:09:48,  7.26s/it]


{681865: array([1491716, 2067198, 4257743, 3953930, 1274723, 1130149, 3179905,
        1892043, 1611760, 1195645, 3334728, 4635518,  425206, 3291577,
        3367029, 5895219, 6300419, 5257499, 6245172, 3737726, 4709751,
        5300984,  845663, 6215240, 2172228, 2653109, 2852476, 2350993,
        3921720,  141847, 2382421, 5708277,   75935, 1994891, 3482688,
        2616438,  862444, 4761216,  944286, 3926343, 1535953, 5047061,
         989427, 3840902, 5407677, 4105697, 2057128, 3743603,  590829,
        3818139, 2587370, 6297202,  401601, 5095625, 1523063, 3174122,
        4575560, 2693539,  237715, 3866277, 3635012, 4363141,  832969,
        4244519, 2567616, 3437596, 2035583, 2616734, 3041239, 2560401,
        1668889,  156987, 1390811, 1965016, 2006793, 1280536, 4198036,
        5802472, 3008882, 4811042, 5763435, 3481290, 2071749, 4990713,
        4442135,  793370, 4237815, 1009134, 2658720, 2446481, 2821703,
        6146832, 3870328, 4343250, 5126201, 3452019, 3211133, 4371783

In [ ]:
from pathlib import Path
user_decoder = joblib.load("models/ordinal_enc_users.joblib")
item_decoder = joblib.load("models/label_enc_items.joblib")



def decode_items(encoded_list):
        return item_decoder.inverse_transform(encoded_list)

def decode_and_merge(pred_df: pd.DataFrame, input_file: str) -> dict:
    # user_ids = user_decoder.categories_[0]
    decoded_users = user_decoder.inverse_transform(pred_df.index.to_numpy().reshape(-1,1)).ravel()
    # print(decoded_users)
    values = np.concatenate(pred_df["values"].to_numpy())
    # display(values)
    decoded = item_decoder.inverse_transform(values).reshape(-1,100)
    # print(decoded.shape)
    
    df_decoded = pd.DataFrame({"user_id": decoded_users,
                            "values": [row for row in decoded],
                               })
    # display(decoded)
    # decoded_items = [decode_items(row) for row in tqdm(pred_df["values"], desc="decoding items")]
    # decoded_items = pred_df["values"].map(item_decoder.inverse_transform)
    # decoded_df = pd.DataFrame({"user_id": decoded_users, "items": decoded_items})
    
    extra_df = pd.read_csv(input_file, header=None)
    extra_df[1] =  extra_df[1].map(lambda x: np.array(list(map(int, x[1:-1].split()))))
    extra_df.columns = ["user_id", "values"]
    # merged_df = extra_df.merge(df_decoded, on="user_id", how="inner")
    merged_df = pd.concat([df_decoded, extra_df])
    # display(merged_df)

    
    result_dict = dict(zip(merged_df["user_id"], merged_df["values"]))
    
    return result_dict

decoded = decode_and_merge(df, 'unknown_users.csv')

def create_submission(recommendations, filename='submission5.csv'):

    print(f"Создаем файл submission: {filename}")
    
    submission_data = []
    for user_id, items in tqdm(recommendations.items(), desc="Формирование submission"):
        submission_data.append({
            'user_id': user_id,
            'item_id_1 item_id_2 ... item_id_100': ' '.join(map(str, items))
        })
    
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(filename, index=False)
    
    print(f"Сохранен submission с {len(submission_df):,} пользователями")
    print(f"Размер файла: {Path(filename).stat().st_size / 1024 / 1024:.1f} MB")
    
    return filename

submission_file = create_submission(decoded)

array([1491716, 2067198, 4257743, ..., 4360603, 5980235, 3936378],
      dtype=int64)

(470106, 100)


array([[ 79768930, 110502882, 227745388, ..., 233860843,  62393953,
         25593993],
       [ 60412644,  79768930,  50498923, ..., 110750216,  80364545,
         57188708],
       [272664684, 228164505,  32839218, ..., 152017730, 332411104,
         97170938],
       ...,
       [180028315, 209761699, 244785589, ..., 286518201, 245346955,
           850424],
       [ 78064417, 325249842, 189943276, ..., 336330346, 201962874,
        136436151],
       [ 77696741,  78064417, 225886286, ..., 233257714, 320029651,
        210548040]])

,user_id,values
0,3442691,"[79768930, 110502882, 227745388, 211482320, 68..."
1,1561330,"[60412644, 79768930, 50498923, 120977983, 2277..."
2,1523400,"[272664684, 228164505, 32839218, 324977785, 31..."
3,3636261,"[132029947, 227449318, 91786342, 86554911, 220..."
4,2890631,"[263577203, 77696741, 247423473, 63987378, 162..."
...,...,...
236,401941,"[51974017, 187052809, 207631139, 143497612, 11..."
237,4718730,"[51974017, 187052809, 207631139, 143497612, 11..."
238,4834541,"[51974017, 187052809, 207631139, 143497612, 11..."
239,4002071,"[51974017, 187052809, 207631139, 143497612, 11..."


Создаем файл submission: submission5.csv


Формирование submission: 100%|██████████| 470347/470347 [00:15<00:00, 29603.58it/s]


Сохранен submission с 470,347 пользователями
Размер файла: 437.7 MB
